#### Project: Feedback summarizer

In this project, we analyze the feedback of Amazon products.

In [21]:
# %pip install langchain

Before you run this, please install [Ollama](https://ollama.com/download) and run
`ollama pull` and `ollama pull llama2`

#### Imports

In [22]:
from IPython.display import display, Markdown
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import Ollama

from common_functions import datasets_dir

#### Data collection

In [23]:
import requests
import os
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

def scrape_amazon_reviews(link):
	
	# get product ID from link
	product_id = link.split('/')[-1].split('?')[0]
	
	# ensure no spaces in product ID
	product_id = product_id.replace(' ', '')
	
	filename = f'reviews_{product_id}.csv'
	filepath = os.path.join(datasets_dir, filename)
	# if file exists, return the reviews
	if os.path.exists(filepath):
		return pd.read_csv(filepath)
	
	options = webdriver.ChromeOptions()
	options.add_argument('headless')  # to run the browser in background

	driver = webdriver.Chrome(options=options)
	driver.get(link)
	soup = BeautifulSoup(driver.page_source, "html.parser")
	driver.quit()
	reviews = []

	for review in soup.find_all('div', {'data-hook': 'review'}):
		title = review.find('a', {'data-hook': 'review-title'}).span.text
		# rating = review.find('i', {'data-hook': 'review-star-rating'}).span.text
		body = review.find('span', {'data-hook': 'review-body'}).span.text
		feedback = title + ' ' + body
		# remove new lines
		feedback = feedback.replace('\n', ' ')

		reviews.append({
			# 'rating': rating,
			'feedback': feedback,
		})

	reviews_df = pd.DataFrame(reviews)
	reviews_df.to_csv(filepath, index=False)
	return reviews_df

amazon_in_link = 'https://www.amazon.in/dp/B00KXULGJQ?th=1'
reviews_df = scrape_amazon_reviews(amazon_in_link)
reviews_df.head()

,feedback
0,I recently purchased the TP-Link AC750 WiFi Ra...
1,I have a range issue in the second floor as th...
2,"Excellent product, does the job very well and ..."
3,I purchased a TP Link RE450 (AC1750) Wi-Fi ext...
4,The RE450 is an 802.11ac range extender with a...


In [24]:
# convert reviews into single string
reviews = ' '.join(reviews_df['feedback'].tolist())
reviews[:100]

'I recently purchased the TP-Link AC750 WiFi Range Extender, and I must say, it has made a significan'

#### Creating the model

In [25]:
llm = Ollama(model="llama2")
llm

Ollama()

Define the prompt

In [26]:
# prompt can also be saved to a file and used as a template
prompt = """
Amazon reviews of a product are given below. Give me insights into what should be improved in the product based on the reviews.
"""

prompt = prompt.strip().replace("\n", " ")

In [27]:
CHARACTER_LIMIT = 2000

promptTemplate = ChatPromptTemplate.from_messages([
	("system", prompt),
	("system", reviews[:CHARACTER_LIMIT]),
	("user", "{question}"),
])
chain = promptTemplate | llm | StrOutputParser()  # Parse output as string
# Different operations are chained together to form a 'pipeline'.
# The output of one operation is passed as input to the next.

result = chain.invoke({ "question": "Please provide insights based on the reviews." })

display(Markdown(f"**Insights based on the reviews:**\n\n{result}"))

**Insights based on the reviews:**

Based on the reviews provided, here are some insights into what could be improved in the TP-Link AC750 WiFi Range Extender:

1. Ethernet port issue: Some users have reported a strange issue with the Ethernet port on the RE450 range extender. This may indicate a design flaw or manufacturing defect that needs to be addressed. TP-Link could improve the quality control process to avoid such issues in future products.
2. Range expansion: While the AC750 range extender effectively expanded the coverage of the WiFi network, some users reported that the signal strength decreased with distance from the main router. This suggests that the device may not provide adequate range extension, especially for larger homes or apartments. TP-Link could consider upgrading the antenna design or increasing the transmit power to improve the range of the product.
3. Setup ease: While some users found the setup process easy with WPS, others reported difficulties in setting up the device. TP-Link could simplify the setup process by providing more detailed instructions or introducing a more user-friendly interface.
4. Speed: One user reported that the RE450 range extender provided faster speeds than their previous RE200 model. However, some users also noted that the speed on the extension was not as fast as they expected. TP-Link could consider improving the device's hardware specifications or optimizing its software to provide better performance.
5. Durability: There is no mention of any durability issues with the product in the reviews. However, TP-Link could consider using higher-quality materials to enhance the product's lifespan and reliability.
6. Additional features: Some users may appreciate additional features such as a built-in switch or USB port for storing media content. TP-Link could consider adding these features to future designs to provide more value to customers.

In summary, while the AC750 WiFi Range Extender generally receives positive reviews, there are some areas where TP-Link could improve, such as the Ethernet port issue and range expansion. By addressing these concerns, TP-Link can enhance the product's overall performance and customer satisfaction.